In [4]:


import os
import pandas as pd
import numpy as np
import glob
import itertools 
import subprocess
import math

from bayes_opt import BayesianOptimization, UtilityFunction


pbounds = {'car_intercept': (-5, 5),\
         'walk_transit_intercept': (-5, 5),\
         'drive_transit_intercept': (-5, 5),\
         'ride_hail_transit_intercept': (-5, 5),\
         'ride_hail_intercept': (-5, 5),\
         'ride_hail_pooled_intercept': (-5, 5), \
         'walk_intercept': (-5, 5),\
         'bike_intercept': (-5, 5)}

Repo='/home/berkeleylab/Repository/beam/'
Current='/home/berkeleylab/Calib_documentation/TUNE_integrate/TUNE_test'



In [5]:


def ext_change(param):    
    if param == 'edit':
        for filename in glob.iglob(os.path.join(Repo, 'test/input/beamville/beam.conf')):
            os.rename(filename, filename[:-9] + 'beam.txt')
    elif param == 'save':
        for filename in glob.iglob(os.path.join(Repo, 'test/input/beamville/beam.txt')):
            os.rename(filename, filename[:-8] + 'beam.conf')

def change_conf(hyperparams):
    
    with open(Repo+'test/input/beamville/beam.txt', 'r') as fin:
        file_text=fin.readlines()
        
    for i in range(24,32,1):
        file_text[i] = file_text[i].split('=',1)[0]+'= '
    
    file_text[24] = file_text[24]+str(hyperparams[1])
    file_text[25] = file_text[25]+str(hyperparams[7])
    file_text[26] = file_text[26]+str(hyperparams[2])
    file_text[27] = file_text[27]+str(hyperparams[5])
    file_text[28] = file_text[28]+str(hyperparams[3])
    file_text[29] = file_text[29]+str(hyperparams[4])
    file_text[30] = file_text[30]+str(hyperparams[6])
    file_text[31] = file_text[31]+str(hyperparams[0])
    
    for j in range(24,32,1):
        file_text[j] = file_text[j]+' \n'
        
    with open(Repo+'test/input/beamville/beam.txt', 'w') as fini:
        for i in file_text:
            fini.write(i)
        
      
def run_beam():
    os.chdir(Repo)
    subprocess.call([Repo+'runme.sh'])
    os.chdir(Current)

def all_subdirs_of(b='/home/berkeleylab/Repository/beam/output/beamville/'):
  result = []
  for d in os.listdir(b):
    bd = os.path.join(b, d)
    if os.path.isdir(bd): result.append(bd)
  return result

def beamville(hyperparams):
    ext_change('edit')
    change_conf(hyperparams)
    ext_change('save') 
    run_beam()
    latest_subdir = max(all_subdirs_of(), key=os.path.getmtime)
    df = pd.read_csv(latest_subdir+"/referenceRealizedModeChoice.csv").iloc[[0, -1]].drop(['iterations'], axis=1)
    acc = (df.iloc[0] - df.iloc[-1]).abs().sum()/df.iloc[0].sum()
    return -acc


In [3]:



# Early stopping


max_iter = 81 # maximum iterations per configurations
eta = 3 # downsampling rate
logeta = lambda x: math.log(x)/math.log(eta)
s_max = int(logeta(max_iter)) # number of successive halving
B = (s_max+1)*max_iter # total iteration per execution of successive halving



for s in reversed(range(s_max+1)):
    n = int(math.ceil(int(B/max_iter/(s+1))*eta**s))
    r = max_iter*eta**(-s)
    
    T = [get_random_hyperparameter_configuration() for i in range(n)]
    
    for i in range(s+1):
        n_i = n*eta**(-i)
        r_i = r*eta**(i)
        val_losses = [run_then_return_val_loss(num_iters=r_i,hyperparameters=t) for t in T]
        T = [T[i] for i in argsort(val_losses)[0:int(n_i/eta)]]
        
        
    

































405


'\nfor s in reversed(range(s_max+1)):\n    n = int(math.ceil(int(B/max_iter/(s+1))*eta**s))\n    r = max_iter*eta**(-s)\n    \n    T = [get_random_hyperparameter_configuration() for i in range(n)]\n    \n    for i in range(s+1):\n        n_i = n*eta**(-i)\n        r_i = r*eta**(i)\n        val_losses = [run_then_return_val_loss(num_iters=r_i,hyperparameters=t) for t in T]\n        T = [T[i] for i in argsort(val_losses)[0:int(n_i/eta)]]\n        \n        '

In [11]:


# Interface between Early stopping and Bayesian Optimization





In [3]:

optimizer = BayesianOptimization(f=None, pbounds=pbounds,verbose=2, random_state=1)
utility = UtilityFunction(kind='ucb', kappa=2.5, xi= 0.01)
next_point_to_probe = optimizer.suggest(utility)
hyperparams=list(next_point_to_probe.values())
target = beamville(hyperparams)
optimizer.register(params=next_point_to_probe, target=target)

for _ in range(3):
    next_point = optimizer.suggest(utility)
    target = beamville(list(next_point.values()))
    optimizer.register(params=next_point, target= target)
    print(target, next_point)
    
print(optimizer.max)


-0.9590612002376707 {'bike_intercept': 1.2882078314496788, 'car_intercept': 3.4299308421164447, 'drive_transit_intercept': -2.981665038255379, 'ride_hail_intercept': 4.092179864013451, 'ride_hail_pooled_intercept': 3.3396652221109484, 'ride_hail_transit_intercept': 3.239382486785276, 'walk_intercept': 0.2849931989788548, 'walk_transit_intercept': -0.30738047489274933}
-1.705215046391517 {'bike_intercept': -4.999999999737802, 'car_intercept': -5.0, 'drive_transit_intercept': 5.0, 'ride_hail_intercept': -5.0, 'ride_hail_pooled_intercept': -5.0, 'ride_hail_transit_intercept': -4.999999998813871, 'walk_intercept': 5.0, 'walk_transit_intercept': 5.0}
-1.56815210932858 {'bike_intercept': 5.0, 'car_intercept': -5.0, 'drive_transit_intercept': -5.0, 'ride_hail_intercept': -5.0, 'ride_hail_pooled_intercept': 5.0, 'ride_hail_transit_intercept': 5.0, 'walk_intercept': -5.0, 'walk_transit_intercept': -5.0}
{'target': -0.935810594634124, 'params': {'bike_intercept': -0.8297799529742598, 'car_interc